合并decoder transformer层的linear层（dtl）训练数据集（e39_0-6999）,测试数据集（e39_7000-7999)

In [2]:
import numpy as np
import os
import socket

In [3]:
bits = 256
batch_size = 32
S1 = 32 # linear大小为batch_size*S1*S2
whole_train_sam_num = 7000 # 完整的训练集样本数
smaller_train_sam_num = 3000 # 减小内存消耗的训练集样本数
smallerer_train_sam_num = 1000
smallererer_train_sam_num = 500
smallerererer_train_sam_num = 50

test_sam_num = 1000

host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_intermediate = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_intermediate = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1\\'
elif host_name == 'jm-System-Product-Name':
    dir_intermediate = '/data/hdr/transformer_data/intermediate/'
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(bits))
    # dir1 = '/data/hdr/transformer_data/joined/'
else:
    raise NameError("You are running the script in a new computer, please define dir_intermediate")



In [14]:
def join_from_intermediate_j1(dir_intermediate, dir_t, bits, intermediate_name, sam_num, trainortest):#j1代表合并第一维
    #sam_num:合并的样本数;trainortest:合成训练集填"train",测试集填"test"
    linearpath0= os.path.join(dir_intermediate, str(bits), intermediate_name+'_f%i_e39_0.npy' % bits)#例：此处intermediate_name为linear
    linear0 = np.load(linearpath0)
    print("合并第一维前大小：", linear0.shape)
    #把第一维合并
    linear0_join1 = np.reshape(linear0, (-1, linear0.shape[-1]))
    print("合并第一维后大小：", linear0_join1.shape)
    if trainortest == "test":
        add = 7000
    else:
        add = 0
    for i in range(1+add, sam_num+add):
        linearpath1= os.path.join(dir_intermediate, str(bits), intermediate_name+'_f%i_e39_%i.npy' % (bits, i) )
        linear1 = np.load(linearpath1)
        linear1_join1 = np.reshape(linear1, (-1, linear1.shape[-1]))
        if linear1_join1.shape[0]!=1024:
            print("i",str(i),",shape",str(linear1_join1.shape[0]))
        linear0_join1 = np.append(linear0_join1, linear1_join1, axis=0)
    print(linear0_join1.shape)
    np.save(os.path.join(dir_t, '%s_%s_f%i_sam%i.npy' % (intermediate_name,trainortest,bits,sam_num)), linear0_join1) 
    # linear_name = ""
    # if intermediate_name[-2:] == "in":
    #     linear_name=intermediate_name[:-2]
    print("intermediate_name[-3:] == out:",intermediate_name[-3:] == "out")
    if intermediate_name[-3:] == "out":
        
        linear_name=intermediate_name[:-3]
        bias = np.load(os.path.join(dir_train, "%s_b_f%i.npy" % (linear_name, bits)))
        y = linear0_join1 - bias
        np.save(os.path.join(dir_t, '%s_y_%s_f%i_sam%i.npy' % (linear_name, trainortest, bits, sam_num)), y) 
    
        

In [10]:
def join_from_joined(dir_t, intermediate_name, bits, joined_sam_num, sam_num, batch_size, S1, trainortest):# 从已合成训练/测试集中提取更小的训练测试集
    # 例:intermediate_name:'ex_linear1in'
    # joined_sam_num：已合成训练/测试集的样本数
    # sam_num：提取出的训练/测试集的样本数
    linear_whole = np.load(os.path.join(dir_t,  '%s_%s_f%i_sam%i.npy' % (intermediate_name, trainortest, bits,joined_sam_num)))
    print(linear_whole.shape)
    linear_smaller = linear_whole[np.ix_(range(sam_num*batch_size*S1), range(linear_whole.shape[1]))]
    print(linear_smaller.shape)
    np.save(os.path.join(dir_t, '%s_train_f%i_sam%i.npy' % (intermediate_name, bits, sam_num)), linear_smaller) 
    if intermediate_name[-3:] == "out":
        linear_name=intermediate_name[:-3]
        bias = np.load(os.path.join(dir_train, "%s_b_f%i.npy" % (linear_name, bits)))
        y = linear_smaller - bias
        np.save(os.path.join(dir_t, '%s_y_%s_f%i_sam%i.npy' % (linear_name, trainortest, bits, sam_num)), y) 

## 合并dx_linear1训练数据集（e39_0-6999），把第一维合并

In [6]:
join_from_intermediate_j1(dir_intermediate, dir_train, bits, "dx_linear1in", whole_train_sam_num, "train")#j1代表合并第一维

合并第一维前大小： (32, 32, 64)
合并第一维后大小： (1024, 64)
(7168000, 64)


In [7]:
join_from_joined(dir_train, "dx_linear1in", bits, whole_train_sam_num, smaller_train_sam_num, batch_size, S1, "train")

(7168000, 64)
(3072000, 64)


In [8]:
join_from_joined(dir_train, "dx_linear1in", bits, smaller_train_sam_num, smallerer_train_sam_num, batch_size, S1, "train")

(3072000, 64)
(1024000, 64)


In [9]:
join_from_joined(dir_train, "dx_linear1in", bits, smallerer_train_sam_num, smallererer_train_sam_num, batch_size, S1, "train")

(1024000, 64)
(512000, 64)


In [10]:
join_from_joined(dir_train, "dx_linear1in", bits, smallererer_train_sam_num, smallerererer_train_sam_num, batch_size, S1, "train")

(512000, 64)
(51200, 64)


In [11]:
join_from_intermediate_j1(dir_intermediate, dir_train, bits, "dx_linear1out", whole_train_sam_num, "train")#j1代表合并第一维
join_from_joined(dir_train, "dx_linear1out", bits, whole_train_sam_num, smaller_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear1out", bits, smaller_train_sam_num, smallerer_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear1out", bits, smallerer_train_sam_num, smallererer_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear1out", bits, smallererer_train_sam_num, smallerererer_train_sam_num, batch_size, S1, "train")


合并第一维前大小： (32, 32, 128)
合并第一维后大小： (1024, 128)
(7168000, 128)
(7168000, 128)
(3072000, 128)
(3072000, 128)
(1024000, 128)
(1024000, 128)
(512000, 128)
(512000, 128)
(51200, 128)


In [31]:
# bias = np.load(os.path.join(dir_train, "dx_linear1_b_f%i.npy" % bits))
# print(bias.shape)
# dx_linear1_trainpath= os.path.join(dir_train, 'dx_linear1out_train_f%i_sam%i.npy' % (bits, whole_train_sam_num))
# dx_linear1_train = np.load(dx_linear1_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
# y_smaller = dx_linear1_train - bias
# np.save(os.path.join(dir_train, 'dx_linear1_y_train_f%i_sam%i.npy' % (bits, whole_train_sam_num)), y_smaller) 
# # y_smallerer = ex_linear1out_smallerer - bias
# # np.save(os.path.join(dir_train, 'ex_linear1_y_train_f%i_sam%i.npy' % (bits, smallerer_train_sam_num)), y_smallerer) 

(128,)


In [35]:
# dx_linear1_trainpath= os.path.join(dir_train, 'dx_linear1out_train_f%i_sam%i.npy' % (bits, smallerererer_train_sam_num))
# dx_linear1_train = np.load(dx_linear1_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
# y_smaller = dx_linear1_train - bias
# np.save(os.path.join(dir_train, 'dx_linear1_y_train_f%i_sam%i.npy' % (bits, smallerererer_train_sam_num)), y_smaller) 

## 合并ex_linear2训练数据集（e39_0-6999），把第一维合并

In [38]:
join_from_intermediate_j1(dir_intermediate, dir_train, bits, "dx_linear2in", whole_train_sam_num, "train")#j1代表合并第一维

合并第一维前大小： (32, 32, 128)
合并第一维后大小： (1024, 128)
(7168000, 128)


In [40]:
join_from_joined(dir_train, "dx_linear2in", bits, whole_train_sam_num, smaller_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear2in", bits, smaller_train_sam_num, smallerer_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear2in", bits, smallerer_train_sam_num, smallererer_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear2in", bits, smallererer_train_sam_num, smallerererer_train_sam_num, batch_size, S1, "train")

(7168000, 128)
(3072000, 128)
(3072000, 128)
(1024000, 128)
(1024000, 128)
(512000, 128)
(512000, 128)
(51200, 128)


In [46]:
join_from_intermediate_j1(dir_intermediate, dir_train, bits, "dx_linear2out", whole_train_sam_num, "train")#j1代表合并第一维
join_from_joined(dir_train, "dx_linear2out", bits, whole_train_sam_num, smaller_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear2out", bits, smaller_train_sam_num, smallerer_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear2out", bits, smallerer_train_sam_num, smallererer_train_sam_num, batch_size, S1, "train")
join_from_joined(dir_train, "dx_linear2out", bits, smallererer_train_sam_num, smallerererer_train_sam_num, batch_size, S1, "train")


## 合并dx_linear1测试数据集（e39_7000-7999），把第一维合并

In [6]:
join_from_intermediate_j1(dir_intermediate, dir_test, bits, "dx_linear1in", test_sam_num, "test")#j1代表合并第一维

合并第一维前大小： (32, 32, 64)
合并第一维后大小： (1024, 64)
(1024000, 64)


In [15]:
join_from_intermediate_j1(dir_intermediate, dir_test, bits, "dx_linear1out", test_sam_num, "test")#j1代表合并第一维

合并第一维前大小： (32, 32, 128)
合并第一维后大小： (1024, 128)
(1024000, 128)
intermediate_name[-3:] == out: True


## 合并ex_linear2测试数据集（e39_7000-7999），把第一维合并

In [16]:
join_from_intermediate_j1(dir_intermediate, dir_test, bits, "dx_linear2in", test_sam_num, "test")#j1代表合并第一维
join_from_intermediate_j1(dir_intermediate, dir_test, bits, "dx_linear2out", test_sam_num, "test")#j1代表合并第一维


合并第一维前大小： (32, 32, 128)
合并第一维后大小： (1024, 128)
(1024000, 128)
intermediate_name[-3:] == out: False
合并第一维前大小： (32, 32, 64)
合并第一维后大小： (1024, 64)
(1024000, 64)
intermediate_name[-3:] == out: True
